In [2]:
from scipy import linalg
import numpy as np

In [1]:
class Camera(object):
    """表示针孔式照相机的类"""
    
    def __init__(self,P):
        """初始化相机矩阵 P = K[R|t] """
        self.P = P
        self.K = None
        self.R = None
        self.t = None
        self.c = None
        
    def project(self,X):
        """X的投影点，并且进行归一化操作"""
        x = np.dot(self.P,X)
        for i in range(3):
            x[i] /= x[2]
        return x
    
    def factor(self):
        """将相机矩阵P分解为K,R,t，P = K[R|t]"""
        
        #print('P')
        #print(self.P)
        
        #分解前3×3的部分
        K,R = linalg.rq(self.P[:,:3])
        #print('K')
        #print(K)
        #print('R')
        #print(R)
        
        #将K的对角线元素设为正值
        T = np.diag(np.sign(np.diag(K)))
        #print(T)
        #if linalg.det(T) < 0:
            #T[1,1] *= 1
         
        #print(T)
        self.K = np.dot(K,T)
        self.R = np.dot(T,R)
        self.t = np.dot(linalg.inv(self.K),self.P[:,3])
        
        return self.K,self.R,self.t
    
    def center(self):
        """计算并返回照相机的中心C"""
    
        if self.c is not None:
            return self.c
        else:
            #通过因子分解计算c
            self.factor()
            self.c = -np.dot(self.R.T,self.t)
            return self.c
    
    

In [4]:
def rotation_matrix(a):
    """创建一个用于围绕向量a轴旋转的三维旋转矩阵"""
    R = np.eye(4)
    R[:3,:3] = linalg.expm([[0,-a[2],a[1]],[a[2],0,-a[0]],[-a[1],a[0],0]])
    return R
    